## Model

In [ ]:
import os
import numpy as np
import tensorflow as tf
import random
import math

from scipy.stats import ranksums
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import multiply
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence

tf.random.set_seed(123)

In [ ]:
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    filters = max(1, int(channel//ratio))
    shared_layer_one = tf.keras.layers.Dense(filters,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = tf.keras.layers.Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = tf.keras.layers.GlobalAveragePooling2D()(input_feature)    
    avg_pool = tf.keras.layers.Reshape((1,1,channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input_feature)
    max_pool = tf.keras.layers.Reshape((1,1,channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)
   

    cbam_feature = tf.keras.layers.Add()([avg_pool,max_pool])
    cbam_feature = tf.keras.layers.Activation('sigmoid')(cbam_feature)


    return multiply([input_feature, cbam_feature])

In [ ]:
class ACF(layers.Layer):

    def __init__(self, D=1, mid_ch=32, k_row=5, k_col=5, return_center=True, **kwargs):
        super().__init__(**kwargs)
        self.D = D
        self.mid_ch = mid_ch
        self.k_row = k_row
        self.k_col = k_col
        self.return_center = return_center

        self.row_conv = keras.Sequential([
            layers.SeparableConv2D(self.mid_ch, kernel_size=(3, self.k_row),
                                   padding='same', activation='relu'),
            layers.Conv2D(self.mid_ch, kernel_size=1, activation='relu')
        ])

        self.col_conv = keras.Sequential([
            layers.SeparableConv2D(self.mid_ch, kernel_size=(self.k_col, 3),
                                   padding='same', activation='relu'),
            layers.Conv2D(self.mid_ch, kernel_size=1, activation='relu')
        ])

        self.gate_row = keras.Sequential([layers.Conv2D(1, 1, activation='sigmoid')])
        self.gate_col = keras.Sequential([layers.Conv2D(1, 1, activation='sigmoid')])

        self.head = keras.Sequential([
            layers.Conv2D(self.mid_ch, 1, activation='relu'),
            layers.Conv2D(1, 1, activation=None)
        ])

    def _shift(self, x, dy=0, dx=0):
        B, H, W, C = tf.unstack(tf.shape(x))
        pad_top  = tf.maximum( dy, 0)
        pad_bot  = tf.maximum(-dy, 0)
        pad_left = tf.maximum( dx, 0)
        pad_right= tf.maximum(-dx, 0)
        xpad = tf.pad(x, [[0,0],[pad_top,pad_bot],[pad_left,pad_right],[0,0]])
        y0 = tf.maximum(-dy, 0)
        y1 = y0 + H
        x0 = tf.maximum(-dx, 0)
        x1 = x0 + W
        return xpad[:, y0:y1, x0:x1, :]

    def call(self, x, training=None):
        diffs_row = []
        diffs_col = []
        for d in range(1, self.D+1):
            down = self._shift(x, dy=+d)
            up   = self._shift(x, dy=-d)
            right= self._shift(x, dx=+d)
            left = self._shift(x, dx=-d)

            dr = down - x
            ur = up   - x
            rc = right- x
            lc = left - x

            diffs_row += [dr, ur, tf.abs(dr), tf.abs(ur)]
            diffs_col += [rc, lc, tf.abs(rc), tf.abs(lc)]

        Fr = tf.concat(diffs_row, axis=-1)
        Fc = tf.concat(diffs_col, axis=-1)

        R = self.row_conv(Fr)
        C = self.col_conv(Fc)

        gr = self.gate_row(R)
        gc = self.gate_col(C)

        R = R * gr
        C = C * gc

        fused = tf.concat([R, C], axis=-1)
        P = self.head(fused)

        if self.return_center:
            H = tf.shape(P)[1]
            W = tf.shape(P)[2]
            i = H // 2
            j = W // 2
            center = P[:, i:i+1, j:j+1, :]
            center = tf.reshape(center, [tf.shape(P)[0], 1])
            return center
        else:
            return P


In [ ]:
def cbam_channel_only(x, ratio=8):
    return channel_attention(x, ratio)

def init_model():
    inputs = layers.Input(shape=(15,15,1))

    x = layers.Conv2D(128, 3, padding='same', activation='relu', kernel_initializer='he_normal')(inputs)
    x = layers.MaxPooling2D((3,3), strides=(3,3))(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = cbam_channel_only(x, ratio=8)

    acf_logit = ACF(D=2, mid_ch=32, k_row=5, k_col=5, return_center=True)(x) 

    x = layers.Dense(128, activation=None, kernel_initializer='he_normal')(acf_logit)
    x = layers.PReLU()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64, activation=None, kernel_initializer='he_normal')(x)
    x = layers.PReLU()(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01),
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
        metrics=[keras.metrics.TruePositives(name='tp'),
                 keras.metrics.FalsePositives(name='fp'),
                 keras.metrics.TrueNegatives(name='tn'),
                 keras.metrics.FalseNegatives(name='fn'),
                 keras.metrics.BinaryAccuracy(name='accuracy'),
                 keras.metrics.Precision(name='precision'),
                 keras.metrics.Recall(name='recall')]
    )
    return model



model = init_model()
model.summary()

In [ ]:
def fun(a):
    oshape = a.shape
    a = a.reshape(-1,15).astype('float32')
    mean = np.mean(a,axis = 1).reshape(-1,1)
    a = a - mean
    sqrt = (np.sqrt(a.var(axis =1))+1e-10).reshape(-1,1)
    a = a/sqrt
    return a.reshape(oshape)
    
class load_testdata(Sequence):
    def __init__(self, x_y_set, batch_size):
        self.x_y_set = x_y_set
        self.x = self.x_y_set[:,:]
        self.batch_size = batch_size

    def __len__(self):
        
        return math.floor(len(self.x) / self.batch_size)
        
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        batch_x = batch_x.reshape(-1,15,15,1)
           
        return np.array(batch_x)

## Prediction

In [ ]:
test_dir  = '/test/dataset/directory'
model_path= 'path/to/model.weights.h5'
save_dir  = 'path/to/predict_result'
os.makedirs(save_dir, exist_ok=True)

model = init_model()
model.load_weights(model_path)

for fname in sorted(os.listdir(test_dir)):
    if not fname.endswith('.npy'):
        continue
    fpath = os.path.join(test_dir, fname)
    pre_data = np.load(fpath)

    random.seed(0)
    np.random.shuffle(pre_data)

    data1 = fun(pre_data[:, :-1])
    data  = pre_data[:, :-1]

    batch_size = 32
    preprocessed_data = load_testdata(data1, batch_size)
    epo = math.floor(len(pre_data) / batch_size)

    predictions = model.predict(preprocessed_data, batch_size=batch_size, verbose=0)
    predictions = np.array(predictions.flatten()).reshape(-1, 1)

    merged_data = np.concatenate((data[:batch_size * epo], predictions), axis=1)
    merged_data[:, -1] = (merged_data[:, -1] >= 0.5).astype(int)

    save_path = os.path.join(save_dir, fname.replace('.npy', '_pred.npy'))
    np.save(save_path, merged_data)



In [ ]:
def find_submatrix(big_matrix, small_matrix):
    big_rows, big_cols = big_matrix.shape
    small_rows, small_cols = small_matrix.shape
    for i in range(big_rows - small_rows + 1):
        if np.allclose(big_matrix[i:i + small_rows, i:i + small_cols], small_matrix):
            return i + 7, i + 8
    return None

pred_dir = '/path/to/predict_result'
save_dir = '/path/to/predict_results_boundary'
os.makedirs(save_dir, exist_ok=True)

for fname in sorted(os.listdir(pred_dir)):
    if not fname.endswith('_pred.npy'):
        continue

    pred_data = np.load(os.path.join(pred_dir, fname))
    positive_data = pred_data[pred_data[:, -1] == 1, :225]

    base_name = fname.replace('_pred.npy', '.txt')
    matrix_path = os.path.join('/path/to/test/hic/matrix/txt/file', base_name)
    big_matrix = np.loadtxt(matrix_path)

    results = []
    for i in range(positive_data.shape[0]):
        small_matrix = positive_data[i].reshape((15, 15))
        result = find_submatrix(big_matrix, small_matrix)
        if result is not None:
            results.append(result)

    sorted_results = sorted(results)
    with open(os.path.join(save_dir, fname.replace('_pred.npy', '_bound.txt')), 'w') as f:
        for r in sorted_results:
            f.write(f"{r[0]}\t{r[1]}\n")


In [ ]:
# count TAD boundary bins
from pathlib import Path

def count_pos_lastcol(dir_path):
    dirp = Path(dir_path)
    total = 0
    for f in sorted(dirp.glob("*.npy")):
        arr = np.load(f)
        cnt = int(np.sum(arr[:, -1] == 1))
        print(f"{f.name}: {cnt}")
        total += cnt
    print("-" * 40)
    print(f"TOTAL: {total}")

count_pos_lastcol("/path/to/predict_result")


## false positive filtering

In [ ]:
def load_matrix(matrix_path):
    return np.loadtxt(matrix_path)

def parse_prediction(pred_path):
    data = np.loadtxt(pred_path, dtype=int)
    return data[:, 0] if data.ndim == 2 else np.array([data[0]])

def find_gap_regions(matrix, window_size):
    n = matrix.shape[0]
    gap = np.zeros(n)
    for i in range(n):
        s = max(0, i - window_size)
        e = min(n, i + window_size + 1)
        if np.sum(matrix[i, s:e]) == 0:
            gap[i] = -0.5
    return np.where(gap == -0.5)[0]

def which_process_regions(rmv_idx, n_bins, min_size=3):
    proc = sorted(set(range(n_bins)) - set(rmv_idx))
    if not proc:
        return []

    regions = []
    start = proc[0]
    for i in range(1, len(proc)):
        if proc[i] - proc[i - 1] > 1:
            if proc[i - 1] - start + 1 >= min_size:
                regions.append((start, proc[i - 1]))
            start = proc[i]
    if proc[-1] - start + 1 >= min_size:
        regions.append((start, proc[-1]))
    return regions

def get_upstream_triangle(matrix, i, size):
    low = max(0, i - size)
    tri = matrix[low:i+1, low:i+1]
    return tri[np.triu_indices(tri.shape[0], k=1)]

def get_downstream_triangle(matrix, i, size):
    n = matrix.shape[0]
    if i >= n - 1:
        return np.array([])
    up = min(n, i + size + 1)
    tri = matrix[i+1:up, i+1:up]
    return tri[np.triu_indices(tri.shape[0], k=1)]

def get_diamond_matrix(matrix, i, size):
    n = matrix.shape[0]
    new = np.full((size, size), np.nan)
    for k in range(size):
        row_idx = i - k
        if row_idx < 0 or i >= n:
            continue
        l = min(i + 1, n)
        u = min(i + size + 1, n)
        new[size - k - 1, :u - l] = matrix[row_idx, l:u]
    return new.flatten()

def compute_pvalues(matrix, region, size):
    pvalues = np.ones(matrix.shape[0])
    for i in range(region[0], region[1] + 1):
        diamond = get_diamond_matrix(matrix, i, size)
        upstream = get_upstream_triangle(matrix, i, size)
        downstream = get_downstream_triangle(matrix, i, size)
        compare = np.concatenate([upstream, downstream])
        if len(diamond) == 0 or len(compare) == 0:
            continue
        _, p = ranksums(diamond[~np.isnan(diamond)], compare)
        pvalues[i] = p
    return pvalues

def filter_predictions(matrix_path, pred_path, output_path, window_size=8):
    matrix = load_matrix(matrix_path)
    pred_idx = parse_prediction(pred_path)
    n_bins = matrix.shape[0]

    local_ext = np.full(n_bins, 0.0)
    local_ext[pred_idx] = -1

    gap_idx = find_gap_regions(matrix, window_size)
    proc_regions = which_process_regions(gap_idx, n_bins)

    scaled = matrix.copy()
    for d in range(1, 2 * window_size + 1):
        row_idx = np.arange(matrix.shape[0] - d)
        col_idx = row_idx + d
        values = scaled[row_idx, col_idx]
        mean = np.mean(values)
        std = np.std(values) + 1e-10
        scaled[row_idx, col_idx] = (values - mean) / std

    pvalues = np.ones(n_bins)
    for region in proc_regions:
        p = compute_pvalues(scaled, region, window_size)
        pvalues[region[0]:region[1]+1] = p[region[0]:region[1]+1]

    final_ext = np.copy(local_ext)
    for i in range(n_bins):
        if local_ext[i] == -1 and pvalues[i] < 0.05:
            final_ext[i] = -1
        elif local_ext[i] == -1:
            final_ext[i] = 0

    now_local = np.where(final_ext == -1)[0]
    # 인접한 boundary가 있다면 하나만 남김
    if len(now_local) >= 2:
        filtered = []
        skip = False
        for i in range(len(now_local)-1):
            if skip:
                skip = False
                continue
            if now_local[i] + 1 == now_local[i+1]:
                keep = now_local[i] if pvalues[now_local[i]] < pvalues[now_local[i+1]] else now_local[i+1]
                filtered.append(keep)
                skip = True
            else:
                filtered.append(now_local[i])
        if not skip:
            filtered.append(now_local[-1])
        now_local = sorted(set(filtered))

    results = [(x, x+1) for x in now_local]
    with open(output_path, 'w') as f:
        for a, b in results:
            f.write(f"{a}\t{b}\n")


In [ ]:
# === 전체 파일 반복 수행 (flat) ===
pred_dir = "/path/to/predict_results_boundary"
save_dir = "/path/to/final_filtered"
os.makedirs(save_dir, exist_ok=True)

for fname in sorted(os.listdir(pred_dir)):
    if not fname.endswith("_bound.txt"):
        continue
    matrix_fname = fname.replace("_bound.txt", ".txt")
    matrix_path = os.path.join("/path/to/test/hic/matrix/txt/file", matrix_fname)
    pred_path = os.path.join(pred_dir, fname)
    output_path = os.path.join(save_dir, fname.replace("_bound.txt", "_filtered.txt"))

    print(f"[flat] Filtering {fname}")
    filter_predictions(matrix_path, pred_path, output_path, window_size=8)
